# Images

In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

### Loading images

In [ ]:
dataset = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [ ]:
for f in dataset.take(5):
  print(f.numpy())

In [ ]:
def load_image(path):
    img_height = 180
    img_width = 180
    binary_format = tf.io.read_file(path)
    image = tf.image.decode_jpeg(binary_format, channels=3)
    return tf.image.resize(image, [img_height, img_width])

In [ ]:
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.cache().shuffle(buffer_size=1000) # cache only if the dataset fits in memory
dataset = dataset.batch(2)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
for f in dataset.take(5):
  print(f.numpy().shape)

In [ ]:
images = next(iter(dataset))
images.shape

### Filters

Filters are 3-dimensional tensors. Tensorflow stores the different filter weights for a given pixel and channel in the last dimension. Therefore, the structure of a tensor of filters is:

```python
[rows, columns, channels, filters]
```

where channels are the filters in the input thensor for a given layer.

In [ ]:
hfilter = tf.stack([tf.stack([tf.zeros(3), tf.ones(3), tf.zeros(3)]) for _ in range(3)])
hfilter

In [ ]:
vfilter = tf.transpose(hfilter, [0, 2, 1])
vfilter

Given that the values of each filter (for a concrete pixel and channel) are in the last axis, we are goint to stack both filters in the last axis.

In [ ]:
filters = tf.stack([hfilter, vfilter], axis=-1)
filters.shape

In [ ]:
outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")

In [ ]:
plt.figure(figsize=(20,60))
ax = plt.subplot(1, 3, 1)
plt.axis("off")
plt.imshow(images[1].numpy().astype("uint8"))
for i in range(2):
  ax = plt.subplot(1, 3, i + 2)
  plt.imshow(outputs[1, :, :, i], cmap="gray")
  plt.axis("off")

### Pooling

In [ ]:
outputs = tf.nn.max_pool(images, ksize=(1,2,2,1), strides=(1,2,2,1), padding='SAME')
images.shape, outputs.shape

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(2):
  ax = plt.subplot(2, 2, i*2 + 1)
  plt.imshow(images[i, :, :, i], cmap="gray")
  plt.axis("off")
  ax = plt.subplot(2, 2, i*2 + 2)
  plt.imshow(outputs[i, :, :, i], cmap="gray")
  plt.axis("off")

###  Depthwise pooling

Pooling along all the channels for each pixel.

In [ ]:
outputs = tf.nn.max_pool(images, ksize=(1,1,1,3), strides=(1,1,1,3), padding='SAME')
images.shape, outputs.shape

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(2):
  ax = plt.subplot(2, 2, i*2 + 1)
  plt.imshow(images[i, :, :, i], cmap="gray")
  plt.axis("off")
  ax = plt.subplot(2, 2, i*2 + 2)
  plt.imshow(outputs[i, :, :, 0], cmap="gray")
  plt.axis("off")

## Keras vs Tensorflow

In [ ]:
import os
import numpy as np

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [ ]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)

In [ ]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [ ]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [ ]:
get_label(b'/Users/nerea/.keras/datasets/flower_photos/tulips/8686332852_c6dcb2e86b.jpg').numpy()

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [ ]:
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.title(class_names[label])
  plt.axis("off")

### Keras

In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  #layers.MaxPooling2D(),
  layers.GlobalAvgPool2D(),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

In [ ]:
model.summary()

### Tensorflow

#### Model Implementation 

In [ ]:
images = tf.divide(images, 255.)

In [ ]:
shape = tf.TensorShape([3,3,3,32])

In [ ]:
filters_1 = tf.Variable(
    initial_value=tf.initializers.glorot_uniform()(shape),
    shape=shape,
    name='filters_1',
    dtype=tf.float32,
    trainable=True,
    synchronization=tf.VariableSynchronization.AUTO,
    caching_device=None
)

In [ ]:
feature_maps_1 = tf.nn.leaky_relu(
    tf.nn.conv2d(images, filters_1, strides=[1, 1, 1, 1], padding="SAME"),
    alpha=0.2
)

In [ ]:
pooled_maps_1 = tf.nn.max_pool(
    feature_maps_1,
    ksize=(1,3,3,1),
    strides=(1,3,3,1),
    padding='SAME'
)
pooled_maps_1.shape

In [ ]:
shape = tf.TensorShape([3,3,32,32])

In [ ]:
filters_2 = tf.Variable(
    initial_value=tf.initializers.glorot_uniform()(shape),
    shape=shape,
    name='filters_2',
    dtype=tf.float32,
    trainable=True,
    synchronization=tf.VariableSynchronization.AUTO,
    caching_device=None
)

In [ ]:
feature_maps_2 = tf.nn.leaky_relu(
    tf.nn.conv2d(
        pooled_maps_1,
        filters_2,
        strides=[1, 1, 1, 1],
        padding="SAME"
    ),
    alpha=0.2
)

In [ ]:
pooled_maps_2 = tf.nn.max_pool(
    feature_maps_2,
    ksize=(1,3,3,1),
    strides=(1,3,3,1),
    padding='SAME'
)
pooled_maps_2.shape

In [ ]:
shape = tf.TensorShape([3,3,32,64])

In [ ]:
filters_3 = tf.Variable(
    initial_value=tf.initializers.glorot_uniform()(shape),
    shape=shape,
    name='filters_3',
    dtype=tf.float32,
    trainable=True,
    synchronization=tf.VariableSynchronization.AUTO,
    caching_device=None
)

In [ ]:
feature_maps_3 = tf.nn.leaky_relu(
    tf.nn.conv2d(
        pooled_maps_2,
        filters_3,
        strides=[1, 1, 1, 1],
        padding="SAME"
    ),
    alpha=0.2
)

In [ ]:
feature_maps_3.shape

In [ ]:
pooled_maps_3 = tf.nn.max_pool(
    feature_maps_3,
    ksize=(1,60,60,1),
    strides=(1,60,60,1),
    padding='SAME'
)
pooled_maps_3.shape

In [ ]:
flatten = tf.reshape(
    pooled_maps_3,
    shape=tf.TensorShape((2, 64))
)

In [ ]:
shape = tf.TensorShape([64, 64])

In [ ]:
W_1 = tf.Variable(
    initial_value=tf.initializers.glorot_uniform()(shape),
    shape=shape,
    name='W_1',
    dtype=tf.float32,
    trainable=True,
    synchronization=tf.VariableSynchronization.AUTO,
    caching_device=None
)

In [ ]:
X_1 = tf.nn.dropout(
    tf.nn.leaky_relu(
        tf.matmul(flatten, W_1)
    ),
    rate=0.3
)

In [ ]:
shape = tf.TensorShape([64, 5])

In [ ]:
W_2 = tf.Variable(
    initial_value=tf.initializers.glorot_uniform()(shape),
    shape=shape,
    name='W_2',
    dtype=tf.float32,
    trainable=True,
    synchronization=tf.VariableSynchronization.AUTO,
    caching_device=None
)

In [ ]:
X_2 = tf.nn.dropout(
    tf.nn.leaky_relu(
        tf.matmul(X_1, W_2)
    ),
    rate=0.3
)

In [ ]:
scores = tf.nn.softmax(X_2)

#### Training

In [ ]:
optimizer = tf.optimizers.Adam(0.01)

In [ ]:
num_epochs = 256

for e in range(num_epochs):
    for img, label in dataset:
        with tf.GradientTape() as tape:
            current_loss = tf.losses.categorical_crossentropy(target, pred)
        grads = tape.gradient(current_loss, weights)
        optimizer.apply_gradients(zip(grads, weights))
        print(tf.reduce_mean(current_loss))